In [ ]:
import pandas as pd

In [ ]:
!pip install nilearn 
!pip install transforms3d

     |████████████████████████████████| 10.1 MB 19.7 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Created wheel for transforms3d: filename=transforms3d-0.3.1-py3-none-any.whl size=59373 sha256=122e22835ca11733af1254f39154878e58a6454934d1f13671ed55c9a8759a01
  Stored in directory: /root/.cache/pip/wheels/b5/b7/93/8985551f83720ce37548a5b543c75380bb707955a9c2c5d28c
Successfully built transforms3d


In [ ]:
import nibabel as nb

from pathlib import Path
from pprint import pprint


import numpy as np 
from matplotlib import pyplot as plt
from nilearn import plotting as nlp # Neiroimage plotting
import transforms3d
from scipy import ndimage as ndi
import nibabel.testing
from collections import defaultdict

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
def append_label(
    train_info, 
    img_data,
    img_3d,
    xyz, 
    i_xyz, 
    coord_1, 
    coord_2, 
    img_n, 
    annot_info,
    ):
  for i in range(0, img_data.shape[i_xyz]):
    train_info['img_name'].append(f"{img_n}_{xyz}_{i}")
    for label in range(0, 12):
      if label in annot_info.keys():
        if (
            int(annot_info[label][coord_1]) <= i 
            and int(annot_info[label][coord_2]) >= i
            ):
          train_info[label].append(1)
        else:
          train_info[label].append(0)
      else:
        train_info[label].append(0)

  return train_info

In [ ]:
# training
indexes = range(0, 28)
train_info = defaultdict(list)
# img_dict = {}
for img_num in indexes:
  annot_info = {}
  img_vol_path = f"/content/drive/MyDrive/LITS_all_train/Training Batch 1/volume-{img_num}.nii"
  annot_path = f"/content/drive/MyDrive/LITS_all_train/annotations_of_training_set/segmentation-{img_num}.txt"
  with open(annot_path) as f:
    lines = f.readlines()

  for l in lines:
    l = l.split(' ')
    l[-1] = l[-1][:-1]
    annot_info[l[1]] = l[2:]


  img_data = nb.load(img_vol_path)
  img_3d = img_data.get_fdata()

  train_info = append_label(
      train_info=train_info,
      img_data=img_data,
      img_3d=img_3d, 
      xyz='x', 
      i_xyz=0, 
      coord_1=0, 
      coord_2=1, 
      img_n=img_num, 
      annot_info=annot_info
  )
  train_info = append_label(
      train_info=train_info,
      img_data=img_data,
      img_3d=img_3d, 
      xyz='y',
      i_xyz=1, 
      coord_1=2, 
      coord_2=3, 
      img_n=img_num, 
      annot_info=annot_info
  )
  train_info = append_label(
      train_info=train_info,
      img_data=img_data,
      img_3d=img_3d, 
      xyz='z',
      i_xyz=2, 
      coord_1=4, 
      coord_2=5, 
      img_n=img_num, 
      annot_info=annot_info
  )
# where keys is coordinates, values is list of labels


In [ ]:
pd.DataFrame(data=train_info).to_csv('/content/drive/MyDrive/LITS_all_train/train_batch_i_multilabel.csv')